## Import packages and set up functions

In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from missingpy import MissForest

RSEED = 26

# Class for label encoding all data
class LabelEncoderByCol(BaseEstimator, TransformerMixin):
    def __init__(self,col):
        #List of column names in the DataFrame that should be encoded
        self.col = col
        #Dictionary storing a LabelEncoder for each column
        self.le_dic = {}
        for el in self.col:
            self.le_dic[el] = LabelEncoder()

    def fit(self,x,y=None):
        #Fill missing values with the string 'NaN'
        x[self.col] = x[self.col].fillna('NaN')
        for el in self.col:
            #Only use the values that are not 'NaN' to fit the Encoder
            a = x[el][x[el]!='NaN']
            self.le_dic[el].fit(a)
        return self

    def transform(self,x,y=None):
        #Fill missing values with the string 'NaN'
        x[self.col] = x[self.col].fillna('NaN')
        for el in self.col:
            #Only use the values that are not 'NaN' to fit the Encoder
            a = x[el][x[el]!='NaN']
            #Store an ndarray of the current column
            b = x[el].to_numpy()
            #Replace the elements in the ndarray that are not 'NaN'
            #using the transformer
            b[b!='NaN'] = self.le_dic[el].transform(a)
            #Overwrite the column in the DataFrame
            x[el]=b
        #return the transformed DataFrame
        return x



## Import data and set up

In [92]:
gen_data = pd.read_csv('C:/Users/samuel/Google Drive/Elise Projects/Genomics/Data/gen_data_new_split.csv')

gen_data = gen_data[gen_data.columns[1:]]

y_col = gen_data['X7m_con'].copy()
y_col.loc[y_col == 'NaN'] = np.nan
gen_data = gen_data.loc[:, ['X1800544', 'X6277', 'X4680', 'X4354185', 'X1076560', 'X1800955', 'X907094', 'X242446']]

# Label encode data
encoder = LabelEncoderByCol(gen_data.columns)
encoder.fit(gen_data)
encoder.transform(gen_data)

# Impute NaN using random forest
imputer = MissForest()
X_imputed = imputer.fit_transform(gen_data, cat_vars=list(range(8)))
data_imputed = pd.DataFrame(X_imputed, columns = gen_data.columns)
data_imputed['Behavioral'] = y_col
data_imputed = data_imputed.dropna()
data_imputed = data_imputed.apply(pd.to_numeric, downcast='integer')

C:\Users\samuel\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [93]:
X_train, X_test, y_train, y_test = train_test_split(data_imputed[data_imputed.columns[0:8]], 
                                                    data_imputed[data_imputed.columns[-1]], test_size=0.33, random_state=RSEED)

## Set up model

In [97]:
from sklearn.ensemble import RandomForestRegressor

# Create the model with 100 trees
model = RandomForestRegressor(n_estimators = 100,
                              min_samples_leaf = 4,
                              bootstrap = True,
                              random_state=RSEED)

# Fit on training data
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=26, verbose=0,
                      warm_start=False)

## Feature selection through RFECV

In [95]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

rfecv = RFECV(estimator=model, 
              step=1, 
              cv=KFold(3),
              n_jobs=-1, scoring='r2')

rfecv.fit(X_train, y_train)

RFECV(cv=KFold(n_splits=50, random_state=None, shuffle=False),
      estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                      max_depth=None, max_features='auto',
                                      max_leaf_nodes=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=4, min_samples_split=2,
                                      min_weight_fraction_leaf=0.0,
                                      n_estimators=100, n_jobs=None,
                                      oob_score=False, random_state=26,
                                      verbose=0, warm_start=False),
      min_features_to_select=1, n_jobs=-1, scoring='r2', step=1, verbose=0)

In [96]:
X_train.drop(X_train.columns[np.where(rfecv.support_ == False)[0]], axis=1, inplace=True)
X_test.drop(X_test.columns[np.where(rfecv.support_ == False)[0]], axis=1, inplace=True)

print(X_train.columns)

Index(['X1800955'], dtype='object')


## Test model predictions

In [103]:
from sklearn.model_selection import cross_val_score

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

cv_score = cross_val_score(model, X, y, cv=3, scoring='r2')

print(cv_score.mean())

0.02144602043585035


## Hyperparameter tuning

In [73]:
from sklearn.model_selection import GridSearchCV

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 2000, num = 20)]

# number of features at every split
max_features = [None, 'auto', 'sqrt', 'log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Random search of parameters
model_rand = GridSearchCV(estimator = model, 
                                param_grid = param_grid, 
                                cv = 3, verbose=2, n_jobs = -1,
                                scoring='r2')

# Fit the model
model_rand.fit(X_train, y_train)

# print results
print(model_rand.best_params_)

Fitting 3 folds for each of 15840 candidates, totalling 47520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 6837 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 7930 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 9105 tasks      | 

KeyboardInterrupt: 

## Set up tuned model and test

In [58]:
model_2 = model_rand.best_estimator_

model_2.fit(X_train, y_train)

cv_score = cross_val_score(model_2, X, y, cv=3, scoring='r2')

print(cv_score.mean())

0.015699577864448793
